In [20]:
print("En esta ocasión desarrollaremos un agente que ayude en la gestión efectiva del tiempo \
      \n el mismo, medirá la “productividad” del usuario. Dicha “productividad” corresponde \
      \n a la cantidad de tareas realizadas en un tiempo determinado. \
      \n A su vez, el agente le recomendara al usuario acciones a tomar para un mayor provecho del tiempo.")

En esta ocasión desarrollaremos un agente que ayude en la gestión efectiva del tiempo       
 el mismo, medirá la “productividad” del usuario. Dicha “productividad” corresponde       
 a la cantidad de tareas realizadas en un tiempo determinado.       
 A su vez, el agente le recomendara al usuario acciones a tomar para un mayor provecho del tiempo.


In [21]:
print("Como libería, pensamos utilizar MESA de python puesto que en la descripción de la librería \
      \n nos indica que está orientado a horarios de agentes, por lo cual podríamos basarnos \
      \n en algunos de los métodos que ya la librería implementa,  \
      \n sin embargo, es posible que también utilicemos otras liberías como AgentPy \
      \n puesto que en Mesa no estará todo lo que necesitamos ")

Como libería, pensamos utilizar MESA de python puesto que en la descripción de la librería       
 nos indica que está orientado a horarios de agentes, por lo cual podríamos basarnos       
 en algunos de los métodos que ya la librería implementa,        
 sin embargo, es posible que también utilicemos otras liberías como AgentPy       
 puesto que en Mesa no estará todo lo que necesitamos 


In [4]:
print("Estas son las importaciones de las librerías que utilizaremos:")
print("from mesa import Agent, Model")
print("from mesa.time import RandomActivation")
print("from mesa.space import MultiGrid")
print("import agentpy as ap")
print("import ipysimulate as ips")
print("from ipywidgets import AppLayout")
print("from agentpy.examples import WealthModel, SegregationModel")

import numpy
#import mesa
import agentpy
import ipysimulate
import ipywidgets
import pandas as pd

Estas son las importaciones de las librerías que utilizaremos:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
import agentpy as ap
import ipysimulate as ips
from ipywidgets import AppLayout
from agentpy.examples import WealthModel, SegregationModel


In [6]:
#definamos la estructura más básica de nuestro agente
#la cual podrá ir creciendo según se definan nuevas características

class ScheduleAgent:
  def __init__(self, name, age, gender, dailyTask, comments):
    self.name = name
    self.age = age
    self.gender = gender
    self.dailyTask = dailyTask
    self.notification = comments
    self.comments = comments

taskList = pd.read_csv('taskList.csv')
comentarios = ""

my_Agent = ScheduleAgent("Pepito", "20", "Masculino", taskList, comentarios)

In [33]:
# Estructura del árbol con el uso de la librería anytree
# pip install anytree

from anytree import NodeMixin, RenderTree
import pandas as pd

class Task:
    def __init__(self, id, name, avgTime, urgency):
        self.id=id
        self.name=name
        self.avgTime=avgTime
        self.urgency=urgency
    
    def __str__(self):
        return f"Name: {self.name}, AvgTime: {self.avgTime}, Urgency: {self.urgency}"

class TaskClass(Task, NodeMixin):  # Add Node feature
    def __init__(self, name, avgTime, urgency, parent=None, children=None):
        super(Task, self).__init__()
        self.name = name
        self.avgTime = avgTime
        self.urgency = urgency
        self.parent = parent
        if children:
            self.children = children

In [32]:
def getNodesList(taskList:pd.DataFrame) -> list:
    nodeList = []
    for i in range(0, len(taskList["Name"])):
        nodeList.append(TaskClass(
            name = taskList["Name"][i],
            avgTime = taskList["AvgTime"][i],
            urgency = taskList["Urgency"][i]
        ))
    return nodeList



def setParentNodes(taskList:pd.DataFrame, nodeList:list) -> TaskClass:
    for i in range(1, len(nodeList)):
        node = nodeList[i]
        node.parent = nodeList[(int(taskList["Parent"][i]))-1]
    return nodeList[0] #retorno root del arbol ya que desde root lo puedo recorrer todo



def buildTree(taskList:pd.DataFrame) -> TaskClass: #construccion del arbol por medio de archivo CSV
    nodeList = getNodesList(taskList)
    rootNode = setParentNodes(taskList, nodeList)
    return rootNode

In [11]:
def printTree(rootNode: TaskClass) -> None:
    for pre, fill, node in RenderTree(rootNode):
        print("%s%s" % (pre, node.name))    

In [58]:
task_df = pd.read_csv('taskList.csv') 
task_tree = buildTree(task_df)

# print(task_df)
printTree(task_tree)

desayunar
├── bañarse
│   ├── hacer la tarea
│   │   ├── almorzar
│   │   └── hacer ejercicio
│   └── estudiar para el parcial
│       ├── escuchar musica
│       └── leer documentacion
└── sacar al perro
    ├── hacer estiramientos
    │   ├── ver netflix
    │   └── reunirse con amigos
    └── regar plantas
        ├── ordenar el cuarto
        └── jugar con el perro


In [60]:
# root
print("task tree: " + str(task_tree))
print("task tree.name: " + task_tree.name)
print("task tree.children: " + str(task_tree.children))
print("task tree.children left: " + str(task_tree.children[0])) #IZQ
print("task tree.children right: " + str(task_tree.children[1])) #DER
#pp(task_tree.children)
print(type(task_tree.children))
print("task tree.height: " + str(task_tree.height))
print("task tree.root: " + str(task_tree.root))
print("task tree.children left name: " + str(task_tree.children[0].name)) #IZQ
print("task tree.children left name: " + str(task_tree.children[0].children[0])) #IZQ

task tree: Name: desayunar, AvgTime: 40, Urgency: 5
task tree.name: desayunar
task tree.children: (<__main__.TaskClass object at 0x00000195A7EE46D0>, <__main__.TaskClass object at 0x00000195A7EE4340>)
task tree.children left: Name: bañarse, AvgTime: 45, Urgency: 2
task tree.children right: Name: sacar al perro, AvgTime: 30, Urgency: 5
<class 'tuple'>
task tree.height: 3
task tree.root: Name: desayunar, AvgTime: 40, Urgency: 5
task tree.children left name: bañarse
task tree.children left name: Name: hacer la tarea, AvgTime: 60, Urgency: 7


In [53]:
class Choice():
    def __init__(self, move, urgency):
        self.move = move
        self.urgency = urgency

    def __str__(self):
        return self.move + ": " + str(self.urgency)



def miniMax(rootNode, isMax):
    children = rootNode.children
    try:
        l_choice = miniMax(children[0], not isMax)
        r_choice = miniMax(children[1], not isMax)

        if (isMax):
            if (l_choice.urgency > r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency)
        else:
            if (l_choice.urgency < r_choice.urgency):
                return Choice("left", l_choice.urgency)
            else:
                return Choice("right", r_choice.urgency) 
    except IndexError:
        return Choice("end", rootNode.urgency)

In [57]:
rootNode = buildTree(my_Agent.dailyTask) #contruccion del arbol
i = 10
while (i < 10):
    choice = miniMax(rootNode, True)
print(choice)


right: 4


In [25]:
# metodo 2 de grafos "hechos a mano"
# No haremos el árbol así pero si el minmax, hay que adaptar el mixmax al uso del otro árbol, el árbol de anytree

class Node():
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

    def __str__(self):
        return 
        
class Choice():
    def __init__(self, move, value):
        self.move = move
        self.value = value

    def __str__(self):
        return self.move + ": " + str(self.value)
    
# root
root = Node(None)
root.left = body2_1
root.right = body2_2

# 2nd layer
body2_1 = Node(-2)
body2_1.left = body3_1
body2_1.right = body3_2

body2_2 = Node(10)
body2_2.left = body3_3

# 3rd layer
body3_1 = Node(3)
body3_1.left = leaf1
body3_1.right = leaf2

body3_2 = Node(20)
body3_2.left = leaf3
body3_2.right = leaf4

body3_3 = Node(-8)
body3_3.left = leaf5
body3_3.right = leaf6


# 4th layer
leaf1 = Node(-8)
leaf2 = Node(7)
leaf3 = Node(15)
leaf4 = Node(14)
leaf5 = Node(8)
leaf6 = Node(-5)


def minimax(node, is_max):    
    # base case, if no sub nodes, return the value
    try:

        # if child nodes exist, run minimax on each child nodes
        l_choice = minimax(node.left, not is_max)
        r_choice = minimax(node.right, not is_max)
        
        # compare results
        if (is_max):
            if (l_choice.value > r_choice.value):
                return Choice("left", l_choice.value)
            else:
                return Choice("right", r_choice.value)
        else:
            if (l_choice.value < r_choice.value):
                return Choice("left", l_choice.value)
            else:
                return Choice("right", r_choice.value)
        
        
# run core programmmm

# initialize our state
current_node = root
is_max = True


while (True):
    # run minimax on current node
    choice = minimax(current_node, is_max)
    
    # make choice based on minimax search
    if (choice.move == "left"):
        print ("Moving left to node with value " + str(current_node.left.value))
        current_node = current_node.left
    elif (choice.move == "right"):
        print ("Moving right to node with value " + str(current_node.right.value))
        current_node = current_node.right
    elif (choice.move == "end"):
        print ("Game ends with a score of " + str(choice.value))
        break
    
    


NameError: name 'body2_1' is not defined